In [1]:
import pandas as pd
import numpy as np
import xlsxwriter 
from datetime import datetime

### 1、读数据

In [2]:
df=pd.read_excel(r'留存率.xlsx')
df.head(5)           

,日期,注册设备,LTV1,LTV2,LTV3,LTV4,LTV5,LTV6,LTV7,LTV8,...,LTV14,LTV15,LTV16,LTV17,LTV18,LTV19,LTV20,LTV21,LTV22,LTV23
0,20250303(一),156,0.011523,0.016042,0.019171,0.022797,0.025976,0.027962,0.029949,0.031836,...,0.043260,0.045693,0.047829,0.048971,0.050114,0.052249,0.052547,0.053342,0.054633,0.055527
1,20250304(二),12442,0.002829,0.003920,0.004647,0.005167,0.005590,0.006002,0.006317,0.006592,...,0.007761,0.007889,0.008046,0.008203,0.008331,0.008517,0.008724,0.008861,0.009018,0.009175
2,20250305(三),55900,0.002920,0.004045,0.004666,0.005121,0.005493,0.005741,0.005973,0.006171,...,0.007065,0.007172,0.007288,0.007412,0.007536,0.007668,0.007768,0.007867,0.007975,0.008057
3,20250306(四),87133,0.002954,0.003789,0.004289,0.004617,0.004895,0.005118,0.005297,0.005464,...,0.006162,0.006255,0.006366,0.006478,0.006570,0.006663,0.006768,0.006855,0.006941,0.007028
4,20250307(五),111744,0.002968,0.003859,0.004325,0.004639,0.004903,0.005124,0.005321,0.005493,...,0.006212,0.006322,0.006427,0.006543,0.006636,0.006728,0.006820,0.006906,0.006992,0.007078


### 2、数据处理-删除不相关列

In [3]:
# 删除指定列（Cohort Day,Ltv Country），并删除全是缺失值的整列
data=df.drop(columns=["日期"]).dropna(axis=1,how="all") 

### 3、计算加权

In [4]:
# 计算每列缺失值的数量，并显示结果.
na = data.isnull().sum()

# 定义每日留存率函数：weighted，等于每一天的留存人数之和/第一天的登录人数之和，并使结果输出百分号，保留两位小数位
def weighted(i):
    weighted = sum(data.iloc[:data.shape[0]-na[i+1],i+1])/sum(data.iloc[:data.shape[0]-na[i+1],0])*100
    return "%.2f%%" % weighted

# 定义每日留存率函数：weighted_percent，等于每一天的留存率*第一天的登录人数/第一天的登录人数 SUMPRODUCT函数，并使结果输出百分号，保留两位小数位 
def weighted_percent(i):
    weighted_percent = np.sum((data.iloc[:data.shape[0]-na[i+1],i+1])*(data.iloc[:data.shape[0]-na[i+1],0])) /sum(data.iloc[:data.shape[0]-na[i+1],0])*100
    return "%.2f%%" % weighted_percent

In [5]:
input_x = input("请输入数据格式是什么，例如：数值、百分数")

请输入数据格式是什么，例如：数值、百分数数值


In [6]:
weighted_r=[]
if input_x == "数值":
    for i in range(data.shape[1]-1):
        weighted_r.append(weighted(i))
elif input_x == "百分数":
    for i in range(data.shape[1]-1):
        weighted_r.append(weighted_percent(i))
else:
    print("请按照提示输入")

In [7]:
# 增加index一个变量，输出第几天留存率    比如，当i=1,则输出的是第2天留存率
index = []
for i in range(data.shape[1]-1):
    index.append("第"+str(i+1)+"天留存率")
x=index[1]

In [8]:
# 输出结果
weighted_retention_rate = pd.DataFrame(weighted_r,index=index).T

### 4、保存数据结果

In [9]:
# 写入文件
df1=weighted_retention_rate.copy()
df2=df.copy()
writer=pd.ExcelWriter('加权留存率.xlsx')
df1.to_excel(writer,sheet_name='Data1',startcol=0,index=False)#将计算好的加权留存率写入加权留存率输出结果1.xlsx表的Data1工作表
df2.to_excel(writer,sheet_name='sheet1',startcol=0,index=False)#将原始数据写入加权留存率输出结果1.xlsx表的'sheet1'工作表
writer.save()
writer.close()

D:\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:368: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
